In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
import cross_fold
from midrc_dataset import midrc_challenge_dataset
import torchvision.transforms as transforms
import os



# ResNet 50 model 
def create_model():
    model =  torchvision.models.resnet50()
    model.conv1 = torch.nn.Conv2d(1,64, kernel_size=(7,7),stride=(2,2),padding=(3,3), bias=False)
    model.fc = torch.nn.Sequential(torch.nn.Linear(2048, 1, bias=True), torch.nn.Sigmoid())
    return model


In [2]:
def train(epochs,model,device, train_loader, val_loader, criterion, optimizer, fold_number):
    best_vloss = 1_000_000.
    for e in range(epochs):
        print('EPOCH {}:'.format(e + 1))
        running_loss = 0.
        last_loss = 0.
        model.train(True)

        for i, data in enumerate(train_loader):
            # Every data instance is an input + label pair
            inputs, labels = data

            # Zero your gradients for every batch!
            optimizer.zero_grad()

            # Make predictions for this batch
            outputs = model(inputs.to(device))

            # Compute the loss and its gradients
            loss = criterion(outputs*torch.tensor(24.0).to(device), labels.float().to(device).unsqueeze(1))
            loss.backward()

            # Adjust learning weights
            optimizer.step()

            # Gather data and report
            running_loss += loss.item()
            if i % 10 == 9:
                last_loss = running_loss / 10 # loss per batch
                print('  batch {} loss: {}'.format(i + 1, last_loss))
                tb_x = e * len(train_loader) + i + 1
                #tb_writer.add_scalar('Loss/train', last_loss, tb_x)
                running_loss = 0.

        avg_loss = last_loss

        running_vloss = 0.0
        model.eval()
        with torch.no_grad():
            for i, vdata in enumerate(val_loader):
                vinputs, vlabels = vdata
                voutputs = model(vinputs.to(device))
                vloss = criterion(voutputs*torch.tensor(24.0).to(device), vlabels.float().to(device).unsqueeze(1))
                running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = os.path.join('models','modelsave_fold_{}'.format(fold_number))
            torch.save(model.state_dict(), model_path)



In [3]:
def test(model,device,dataloader):
    with torch.no_grad():
        squared_error = 0
        absolute_error = 0

        for data in dataloader:
            inputs, labels = data

            outputs = model(inputs.to(device))

            squared_error += (outputs*torch.tensor(24.0).to(device) - labels.float().to(device).unsqueeze(1))**2
            absolute_error += torch.abs(outputs*torch.tensor(24.0).to(device)-labels.float().to(device).unsqueeze(1))

        #returns mae and rmse
        return absolute_error/len(dataloader), torch.sqrt(squared_error/len(dataloader))

In [4]:
# Training Loop

folds = cross_fold.create_folded_datasets("C:\\Users\\CGarc\\University of Kentucky\\Ahamed, Md. Atik - COVID severity\\data\\resized_224X224\\label_info\\labels.json")

root_dir = 'C:\\Users\\CGarc\\University of Kentucky\\Ahamed, Md. Atik - COVID severity\\data\\resized_224X224'

annotations_file = 'MIDRC mRALE Mastermind Training Annotations_2079_20230428.csv'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 16
epochs = 10

transform = transforms.Compose([
    #    transforms.RandomHorizontalFlip(),
    #    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

for f_i,fold in enumerate(folds):
    train_list, val_list = fold

    model = create_model()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters())

    criterion = torch.nn.MSELoss()

    train_dataset = midrc_challenge_dataset(root_dir, annotations_file, transform, fp_list = train_list)
    val_dataset = midrc_challenge_dataset(root_dir, annotations_file, transform, fp_list = val_list)
    
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True)

    #Training per fold
    train(epochs,model,device,train_loader,val_loader, criterion, optimizer, f_i)    

Fold:  1  Train Indices:  2078  Val indices:  520
Fold:  2  Train Indices:  2096  Val indices:  502
Fold:  3  Train Indices:  2050  Val indices:  548
Fold:  4  Train Indices:  2072  Val indices:  526
Fold:  5  Train Indices:  2096  Val indices:  502
EPOCH 1:
  batch 10 loss: 96.03498115539551
  batch 20 loss: 89.96414623260497
  batch 30 loss: 79.36905670166016
  batch 40 loss: 75.71682624816894
  batch 50 loss: 73.0122745513916
  batch 60 loss: 72.62853469848633
  batch 70 loss: 86.35359954833984
  batch 80 loss: 75.65400733947754
  batch 90 loss: 80.2135612487793
  batch 100 loss: 66.25345649719239
  batch 110 loss: 59.41158485412598
  batch 120 loss: 79.11459732055664
  batch 130 loss: 60.30710868835449
LOSS train 60.30710868835449 valid 117.91435241699219
EPOCH 2:
  batch 10 loss: 73.36745491027833
  batch 20 loss: 61.450362014770505
  batch 30 loss: 74.70352096557617
  batch 40 loss: 81.2291805267334
  batch 50 loss: 86.8306957244873
  batch 60 loss: 66.45353317260742
  batch 70 l

In [5]:
#Testing Loop
for f_i,fold in enumerate(folds):

    train_list, val_list = fold

    val_dataset = midrc_challenge_dataset(root_dir, annotations_file, transform, fp_list = val_list)
    val_loader = DataLoader(val_dataset, batch_size = 1, shuffle=True)

    model = create_model()
    model_pth = os.path.join('models','modelsave_fold_{}'.format(f_i))
    model.load_state_dict(torch.load(model_pth))
    model.to(device)
    model.eval()

    avg_mae, avg_rmse = test(model,device,val_loader)

    print("Fold: ",f_i+1, " MAE:", avg_mae.item() , " RMSE: ", avg_rmse.item())

Fold:  1  MAE: 5.070911407470703  RMSE:  6.121790885925293
Fold:  2  MAE: 5.229495525360107  RMSE:  6.513065814971924
Fold:  3  MAE: 4.875302791595459  RMSE:  6.091127872467041
Fold:  4  MAE: 5.047328472137451  RMSE:  6.208610534667969
Fold:  5  MAE: 4.741214752197266  RMSE:  6.251406192779541
